In [ ]:
#import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import time #there is going to be some sequential logic so importing time
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#set the dataset
beerSales = pd.read_csv('../input/for-simple-exercises-time-series-forecasting/BeerWineLiquor.csv',index_col=0, parse_dates=True)
X = beerSales.index
Y = beerSales['beer'].values.astype(float)
X = np.array(X)

In [ ]:
#split the trainingset and testing set
train_set = Y[:-12]
test_set = Y[-12:]

#scaling
scaler = MinMaxScaler(feature_range=(-1,1))
trainScaler = scaler.fit_transform(train_set.reshape(-1,1))
trainScaler = trainScaler.flatten()
trainScaler = torch.FloatTensor(trainScaler)

In [ ]:
def get_windows(data,ws):
    out = []
    L = len(data)
    for i in range(L-ws):
        out.append((data[i:i+ws],data[i+ws:i+ws+1]))
    return out

window_size = 12
train_data = get_windows(trainScaler,window_size)

In [ ]:
#initiate LSTM model
class LSTM(nn.Module):
    def __init__(self,in_size = 1,hidden_size = 100,out_size = 1):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(in_size,hidden_size)
        self.linear = nn.Linear(hidden_size,out_size)
        self.hidden = (torch.zeros(1,1,self.hidden_size).cuda(),
                       torch.zeros(1,1,self.hidden_size).cuda())
    def forward(self,X):
        lstm_out,self.hidden = self.lstm(X.view(len(X),1,-1),self.hidden)
        pred = self.linear(lstm_out.view(len(X),-1))
        return pred[-1]
model = LSTM().cuda()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)
criterion = nn.MSELoss()
model

In [ ]:
#Train
start = time.time()
epochs = 100
for i in range(epochs):
    for X_train,Y_train in train_data:
        X_train = X_train.cuda()
        Y_train = Y_train.cuda()
        optimizer.zero_grad()
        model.hidden = (torch.zeros(1,1,model.hidden_size).cuda(),
                        torch.zeros(1,1,model.hidden_size).cuda())
        Y_pred = model(X_train)
        loss = criterion(Y_pred,Y_train)
        loss.backward()
        optimizer.step()
    print(f"Epoch : {i+1} LOSS : {loss.item():.7f}")
end = time.time()
dur = end-start
print(f"Duration : {int(dur/60)} minutes and {int(dur%60)} seconds")

In [ ]:
#Testing on the last window size
future = 12
preds = trainScaler[-window_size:].tolist()
model.eval()
for i in range(future):
    X_test = torch.FloatTensor(preds[-window_size:]).cuda()
    with torch.no_grad():
        model.hidden = (torch.zeros(1,1,model.hidden_size).cuda(),
                      torch.zeros(1,1,model.hidden_size).cuda())
        preds.append(model(X_test).item())

In [ ]:
#invertining the normalization
true_predictions = scaler.inverse_transform(np.array(preds[-window_size:]).reshape(-1,1))

#plot
dates = np.arange('2018-02-01', '2019-02-01', dtype='datetime64[M]').astype('datetime64[D]')
plt.figure(figsize=(20,7))
plt.grid(True)
plt.plot(beerSales['beer'],label = 'Original')
plt.plot(dates,true_predictions,label = 'Predicted')
plt.legend()
plt.show()

plt.figure(figsize=(20,7))
plt.grid(True)
plt.plot(beerSales['beer']['2018-01-01':],label = "Original")
plt.plot(dates,true_predictions,label = "Predicted")
plt.legend()
plt.show()